<a href="https://colab.research.google.com/github/mon3/deepNN_NER/blob/master/elmo_test_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q pydrive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
your_module = drive.CreateFile({'id':'1lBy0_BsIQ_gzuCkM-P4MWbstIZFkHj8L'})
your_module.GetContentFile('preprocess.py')


In [0]:
prepro_elmo_module = drive.CreateFile({'id':'1S_fLdw31wQMBtIqIzEIelYlLim4A8dxA'})
prepro_elmo_module.GetContentFile('elmo_preprocessing1.py')

In [0]:
prepro_elmo_module1 = drive.CreateFile({'id':'1jK137fvsVr_DO7UiEA9J0Qbx_LLm7XPS'})
prepro_elmo_module1.GetContentFile('elmo_prep11.py')

In [0]:
validation_module = drive.CreateFile({'id': '1nEox1MVwcpP3Fu5RGJzhKi358x9jTA2h'})
validation_module.GetContentFile('validation.py')

In [0]:
prepro_elmo_module2 = drive.CreateFile({'id':'1gmJJScIBfWmOdDMzOJO2kZvHb-r0zJpG'})
prepro_elmo_module2.GetContentFile('elmo_prep4.py')

In [0]:
prepro_elmo_module3 = drive.CreateFile({'id':'1v9lb4SlclxAAKYjIRk0lrYp5oP3WEU3J'})
prepro_elmo_module3.GetContentFile('elmo_preprocess_sentences_nontrain.py')

In [0]:
"""Load packages"""

import matplotlib.pyplot as plt
import numpy as np
import os

from validation import compute_f1
from keras.models import Model, load_model
from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, \
    Flatten, concatenate, Lambda
#from preprocess import readfile, addCharInformation, padding
from elmo_preprocess_sentences_nontrain import createMatrices, createBatches, createBatchesNonTrain, iterate_minibatches, readfile, addCharInformation, padding
from keras.utils import plot_model
from keras.initializers import RandomUniform
from keras.optimizers import SGD, Nadam
import tensorflow_hub as hub
import tensorflow as tf


Using TensorFlow backend.
W0402 22:12:53.736550 139698042873728 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
"""Set parameters"""

EPOCHS = 80               # paper: 80
DROPOUT = 0.68           # paper: 0.68
DROPOUT_RECURRENT = 0.0  # not specified in paper, 0.25 recommended; in other papers: 0.0
LSTM_STATE_SIZE = 275    # paper: 275
CONV_SIZE = 3             # paper: 3
LEARNING_RATE = 0.0105    # paper 0.0105
OPTIMIZER = Nadam()       # paper uses SGD(lr=self.learning_rate), Nadam() recommended
LOCATION_POINTER = '/content/gdrive/My Drive/masters_thesis/'
CHAR_EMBEDDING = 30   # paper: 25, previously: 30
VERSION = 16
EMBEDDING = "elmo"

Instructions for updating:
Colocations handled automatically by placer.


W0402 22:12:56.036170 139698042873728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
"""Initialise class"""

class CNN_BLSTM(object):
    
    def __init__(self, EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, CHAR_EMBEDDING, VERSION, EMBEDDING):
        
        self.epochs = EPOCHS
        self.dropout = DROPOUT
        self.dropout_recurrent = DROPOUT_RECURRENT
        self.lstm_state_size = LSTM_STATE_SIZE
        self.conv_size = CONV_SIZE
        self.learning_rate = LEARNING_RATE
        self.optimizer = OPTIMIZER
        self.char_embedding_size = CHAR_EMBEDDING
        self.version = VERSION
        self.embedding = EMBEDDING
#         self.tokens_length = []
        
    def loadData(self):
        """Load data and add character information"""
        self.trainSentences = readfile(os.path.join(LOCATION_POINTER, "data/train.txt"))
        self.devSentences = readfile(os.path.join(LOCATION_POINTER, "data/dev.txt"))
        self.testSentences = readfile(os.path.join(LOCATION_POINTER, "data/test.txt"))

    def addCharInfo(self):
        # format: [['EU', ['E', 'U'], 'B-ORG\n'], ...]
        self.trainSentences = addCharInformation(self.trainSentences)
        self.devSentences = addCharInformation(self.devSentences)
        self.testSentences = addCharInformation(self.testSentences)

    def embed(self):
        """Create word- and character-level embeddings"""

        labelSet = set()
        words = {}

        # unique words and labels in data  
        for dataset in [self.trainSentences, self.devSentences, self.testSentences]:
            for sentence in dataset:
                for token, char, label in sentence:
                    # token ... token, char ... list of chars, label ... BIO labels   
                    labelSet.add(label)
                    words[token.lower()] = True
                    
        # mapping for labels
        self.label2Idx = {}
        for label in labelSet:
            self.label2Idx[label] = len(self.label2Idx)

        # mapping for token cases
        case2Idx = {'numeric': 0, 'allLower': 1, 'allUpper': 2, 'initialUpper': 3, 'other': 4, 'mainly_numeric': 5,
                    'contains_digit': 6, 'PADDING_TOKEN': 7}
        # creates identity matrix for token cases
        self.caseEmbeddings = np.identity(len(case2Idx), dtype='float32')  # identity matrix used 

        # read GLoVE word embeddings
        word2Idx = {}
#         self.wordEmbeddings = []
        
#         word represented as 50-dim vector
#         ToDO: test with 300-dim vectors (GloVE 42B, GloVE 84B)
#         if self.embedding == "fastText":
#           fEmbeddings = open(os.path.join(LOCATION_POINTER, "embeddings/wiki-news-300d-1M.vec"), encoding="utf-8")
#         else:
#           fEmbeddings = open(os.path.join(LOCATION_POINTER, "embeddings/glove.6B.50d.txt"), encoding="utf-8")

#         # loop through each word in embeddings
#         for i, line in enumerate(fEmbeddings):
#             if i==0 and self.embedding == "fastText":
#                 continue
                
#             split = line.strip().split(" ") # removes leading and trailing chars and splits into list of single values
#             word = split[0]  # embedding word entry

#             if len(word2Idx) == 0:  # add padding+unknown
#                 word2Idx["PADDING_TOKEN"] = len(word2Idx)
#                 vector = np.zeros(len(split) - 1)  # zero vector for 'PADDING' word
#                 self.wordEmbeddings.append(vector)

#                 word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
#                 vector = np.random.uniform(-0.25, 0.25, len(split) - 1)  # zero vector for 'PADDING' word
#                 self.wordEmbeddings.append(vector)

#             if split[0].lower() in words:
#                 vector = np.array([float(num) for num in split[1:]])
#                 self.wordEmbeddings.append(vector)  # word embedding vector
#                 word2Idx[split[0]] = len(word2Idx)  # corresponding word dict; increments by 1 for each word

#         self.wordEmbeddings = np.array(self.wordEmbeddings)

        # dictionary of all possible characters
        self.char2Idx = {"PADDING": 0, "UNKNOWN": 1}
        for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|<>":
            self.char2Idx[c] = len(self.char2Idx) # 2,3,4 ...

        # format: [[wordindices], [caseindices], [padded word indices], [label indices]]
        self.train_set = padding(createMatrices(self.trainSentences, self.label2Idx, case2Idx, self.char2Idx))
        self.dev_set = padding(createMatrices(self.devSentences, self.label2Idx, case2Idx, self.char2Idx))
        self.test_set = padding(createMatrices(self.testSentences, self.label2Idx, case2Idx, self.char2Idx))

        self.idx2Label = {v: k for k, v in self.label2Idx.items()}  # index to label(reverted)
                                                                
                                                                
    def createBatches(self):
        
        """Create batches"""
        self.train_batch, self.train_batch_len = createBatches(self.train_set)
        self.dev_batch, self.dev_batch_len = createBatches(self.dev_set)
        self.test_batch, self.test_batch_len = createBatches(self.test_set)
#         self.dev_batch, self.dev_batch_len = createBatchesNonTrain(self.dev_set)
#         self.test_batch, self.test_batch_len = createBatchesNonTrain(self.test_set)
        
    def tag_dataset(self, dataset, model):
        """Tag data with numerical values"""
        
        #nTODO; check if labels are numeric
        correctLabels = []
        predLabels = []
        for i in dataset.keys():
          key = str(i)
#           print("I: {}".format(i))
          if int(i) ==1:
            continue
          data = dataset[key]
          tokens = []
          tl = []
          caseing = []
          char = []
          tokens_length = []
          labels = []
          for dt in data:
#             print("DT: {}".format(dt))
            t, c, ch, l, _ = dt
#             if len(c)==1 or len(t)==1:
#               continue
              
#             print("Single token: {}-{}".format(len(c), t))
#             print("Single labels: {}".format(l))
#             if len(t)<=1:
# #               continue
#             print("T: {}".format(t))
#             print(t.split(' '))
#             t = t.split(' ')  # moze tego nie trzeba
          
            t = np.expand_dims(t, -1)
            l = np.expand_dims(l, -1)
#             t = tf.reshape(t, [-1])
      
#             t = np.expand_dims(t.split(' '), -1)
#             c = np.expand_dims(c, -1)
            tokens.append(t)
            caseing.append(c)
            char.append(ch)
            labels.append(l)
            tokens_length.append(tl)
            
          tokens_in = np.asarray(tokens)
          caseing_in = np.asarray(caseing)
          char_in = np.asarray(char)
#           labels_in = np.asarray(labels)
          tokens_len_in = np.asarray(tokens_length)
#           print("TOKENS_IN: {}".format(tokens_in))
#           print("LABELS_IN: {}".format(labels_in))
        
#           print("TAG Tokens size: {}".format(tokens_in.shape))
#           if tokens_in.shape == (1,1):
#             print("Tokens shape (1,1): {}".format(tokens_in))
#             continue
#           print("TAG Labels size: {}".format(labels_in.shape))
#           print("TAG Caseing size: {}".format(caseing_in.shape))
#           print("Tokens: {}".format(tokens_in))


######################## TUTAJ ######################33
          # daje shape (1,1) i się wykrzacza dla arraya z jednym zdaniem
        
  
#           print("Shapes test: {} {} {}".format(tokens_in.shape, caseing_in.shape, char_in.shape))
          if tokens_in.shape[0] >1 and tokens_in.shape[1] >1:
            pred = model.predict([tokens_in, caseing_in, char_in], verbose=False)[0]
            pred = pred.argmax(axis=-1)  # Predict the classes
            correctLabels.append(labels)
            predLabels.append(pred)
          
          
        return predLabels, correctLabels
      
      
      
#         for i, data in enumerate(dataset):
#             tokens, casing, char, labels, _ = data
#             if len(tokens)<=1:
#               continue
#             t = np.expand_dims(tokens, -1)
#             tokens = np.asarray(t)

#             # print("TOKEN: {}".format(t))

#             l = np.expand_dims(labels, -1)
#             labels = np.asarray(l)
            
            
# #             tokens = np.asarray([tokens])
#             print("TAG TOKENS: {}".format(tokens))
#             casing = np.asarray([casing])
#             char = np.asarray([char])
#             pred = model.predict([tokens, casing, char], verbose=False)[0]
#             pred = pred.argmax(axis=-1)  # Predict the classes
#             correctLabels.append(labels)
#             predLabels.append(pred)
#         return predLabels, correctLabels




#         correctLabels = []
#         predLabels = []
#         # ToDO: only training data as sentences
#         # ToDO: change for iteration through dict!
#         for i in dataset.keys():
#           key = str(i)
#           data = dataset[key]
#           for dt in data:
#             ################################################################
#             # sentences or tokens here!?
#             ################################################################
#             tokens, casing, char, labels, _ = dt
# #             tokens = np.asarray([tokens])
#             casing = np.asarray([casing])
#             char = np.asarray([char])
#             print("TAGGED tokens: {}".format(tokens))
#             pred = model.predict([tokens, casing, char], verbose=False, batch_size=len(tokens))[0]
#             pred = pred.argmax(axis=-1)  # Predict the classes
#             correctLabels.append(labels)
#             predLabels.append(pred)
#         return predLabels, correctLabels
    
    

    def ELMoEmbedding(self, tokens_input):
        # ToDO: pojedyncze zdanie na inpucie - lista słów (stringi)
        #       zdania na inpucie - lista list 
        # zdefiniować input jako tokeny!
        """ Return elmo embedding using tf_hub """
        elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
        # signature points to the purpose of why we would like to use the modules
        # as_dict=True needed to output word embeddings instead of defualtss
#         return elmo_model(tf.cast(tokens_input, tf.string), signature="default", as_dict=True)['elmo']  #["word_emb"]  #["default"]  #["word_emb"]

#         return elmo_model(tf.cast(tokens_input, dtype=tf.string), signature="default", as_dict=True)['elmo']  #["word_emb"]  #["default"]  #["word_emb"]
#         return elmo_model(tf.reshape(tf.cast(tokens_input, tf.string), [-1]), signature="default", as_dict=True)['elmo']  #["word_emb"]  #["default"]  #["word_emb"]

        return elmo_model(tf.squeeze(tf.cast(tokens_input, tf.string)), signature="default", as_dict=True)['elmo']  #["word_emb"]  #["default"]  #["word_emb"]
#         return elmo_model(tf.cast(tokens_input, tf.string), signature="default", as_dict=True)['elmo']  #["word_emb"]  #["default"]  #["word_emb"]

#         return elmo_model(inputs={"tokens": tokens_input, "sequence_len": tokens_length}, signature="tokens", as_dict=True)["word_emb"]
#       return elmo_model(x, signature="default", as_dict=True)["word_emb"]
  
    def buildModel(self):
        """Model layers"""

        # character input
        character_input = Input(shape=(None, 52,), name="Character_input")  #input N sentences, each 52 chras length
        embed_char_out = TimeDistributed(
            Embedding(len(self.char2Idx),self.char_embedding_size, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name="Character_embedding")(
            character_input)

        dropout = Dropout(self.dropout)(embed_char_out)

        # CNN
        conv1d_out = TimeDistributed(Conv1D(kernel_size=self.conv_size, filters=30, padding='same', activation='tanh', strides=1), name="Convolution")(dropout)
        maxpool_out = TimeDistributed(MaxPooling1D(52), name="Maxpool")(conv1d_out)  # pool_size=52: max sentence length
        char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
        char = Dropout(self.dropout)(char)

        # word-level input
        # input of N-dimensional vectors (1st arg in shape points to the size of input vectors)
        words_input = Input(shape=(None, ), dtype=tf.string, name='words_input')
        # model takes first dimension as number of examplese and second as the size of each example
#         tokens_length = Input(shape=(None, None, ), name='tokens_length_input')  # co z data type
#         print("After words input")
        # ToDO: Lambda powinna zwrócić odpowiedni rozmiar!
#         words = self.ELMoEmbedding(words_input, tokens_length)
        words = Lambda(self.ELMoEmbedding, output_shape=(None, 1024))(words_input)
#         words_input = Input(shape=(None,), dtype='int32', name='words_input')
#         words = Embedding(input_dim=self.wordEmbeddings.shape[0], output_dim=self.wordEmbeddings.shape[1], weights=[self .wordEmbeddings],
#                           trainable=False)(words_input)  # trainable=False since we provide word embeddings
#         print("After: {}".format(words))

        # case-info input
        casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
        casing = Embedding(output_dim=self.caseEmbeddings.shape[1], input_dim=self.caseEmbeddings.shape[0], weights=[self.caseEmbeddings],
                           trainable=False)(casing_input)  ## trainable=False since we provide case embeddings
       
      
#         print("SIZES FOR CONCATENATION: {} {} {}".format(words.shape, char.shape, casing.shape))
        # concat & BLSTM
#         print("DIMENSIONS: {}".format(tf.shape(words_input)))
              
        output = concatenate([words, casing, char])
#         output = concatenate([tf.reshape(words, 
#                                   [len(words), 52, 1024]), casing, char])
        output = Bidirectional(LSTM(self.lstm_state_size, 
                                    return_sequences=True, 
                                    dropout=self.dropout,                        # on input to each LSTM block
                                    recurrent_dropout=self.dropout_recurrent     # on recurrent input signal
                                   ), name="BLSTM")(output)
        output = TimeDistributed(Dense(len(self.label2Idx), activation='softmax'),name="Softmax_layer")(output)

        # set up model

        self.model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
        
        for layer in self.model.layers:
            print("Layer {}: {}".format(layer.name, layer.output_shape))
        
        
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer=self.optimizer)
        
        self.init_weights = self.model.get_weights()
        
        plot_model(self.model, to_file=os.path.join(LOCATION_POINTER, 'model_{}.png'.format(self.version)))
        
        print("Model built. Saved model.png\n")
        
    def train(self):
        """Default training"""

        self.f1_test_history = []
        self.f1_dev_history = []

        for epoch in range(self.epochs):    
            print("Epoch {}/{}".format(epoch, self.epochs))
            # batch: [word_indices, case_indices, char_indices]
            print("Batch len: {}".format(self.train_batch_len))
            for i,batch in enumerate(iterate_minibatches(self.train_batch, self.train_batch_len)):
                labels, tokens, casing, char, self.tokens_length = batch  
#                 tokens = np.expand_dims(tokens[0], 1)
#                 print(tokens)
#                 print("length: {}".format(len(tokens)))
#                 print("TOKENS: {}".format(tokens))
#                 tokens_in = tf.reshape(tokens, [-1])
#                 print(labels)
#                 print(tokens[:5])
#                 print("before train on batch - i: {}".format(i))
                if len(tokens) <= 1:
                  continue
#                 if i==55 or i ==2:
#                   print(len(tokens), len(labels))
#                 print("Shapes: {} {} {}".format(tokens.shape, casing.shape, char.shape))
                self.model.train_on_batch([tokens, casing, char], labels)
#                 print("after train on batch - i: {}".format(i))


            # compute F1 scores
            predLabels, correctLabels = self.tag_dataset(self.test_batch, self.model)
            pre_test, rec_test, f1_test = compute_f1(predLabels, correctLabels, self.idx2Label)
            self.f1_test_history.append(f1_test)
            print("f1 test ", round(f1_test, 4))

            predLabels, correctLabels = self.tag_dataset(self.dev_batch, self.model)
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, self.idx2Label)
            self.f1_dev_history.append(f1_dev)
            print("f1 dev ", round(f1_dev, 4), "\n")
            
        print("Final F1 test score: ", f1_test)
            
        print("Training finished.")
            
        # save model
        self.modelName = "{}_{}_{}_{}_{}_{}_{}_{}_{}_{}".format(self.epochs, 
                                                        self.dropout, 
                                                        self.dropout_recurrent, 
                                                        self.lstm_state_size,
                                                        self.conv_size,
                                                        self.learning_rate,
                                                        self.char_embedding_size,
                                                        self.optimizer.__class__.__name__,
                                                        self.version,
                                                        self.embedding
                                                       )
        
        modelName = self.modelName + ".h5"
        self.model.save(os.path.join(LOCATION_POINTER, modelName))
        print("Model weights saved.")
        
        self.model.set_weights(self.init_weights)  # clear model
        print("Model weights cleared.")

    def writeToFile(self):
        """Write output to file"""

        output = np.matrix([[int(i) for i in range(self.epochs)], self.f1_test_history, self.f1_dev_history])

        fileName = os.path.join(LOCATION_POINTER, self.modelName + ".txt")
        with open(fileName,'wb') as f:
            for line in output:
                np.savetxt(f, line, fmt='%.5f')
                
        print("Model performance written to file.")
        
    def saveResults(self):
        plt.plot(cnn_blstm.f1_test_history, label = "F1 test")
        plt.plot(cnn_blstm.f1_dev_history, label = "F1 dev")
        plt.xlabel("Epochs")
        plt.ylabel("F1 score")
        plt.legend()
        plt.savefig(os.path.join(LOCATION_POINTER, self.modelName + ".png"))

    print("Class initialised.")

Class initialised.


In [0]:
"""Construct and run model"""

cnn_blstm = CNN_BLSTM(EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, CHAR_EMBEDDING, VERSION, EMBEDDING)
cnn_blstm.loadData()
cnn_blstm.addCharInfo()
cnn_blstm.embed()
cnn_blstm.createBatches()
cnn_blstm.buildModel()
cnn_blstm.train()
cnn_blstm.writeToFile()
cnn_blstm.saveResults()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0402 22:27:39.396083 139698042873728 saver.py:1483] Saver not created because there are no variables in the graph to restore


Layer Character_input: (None, None, 52)
Layer Character_embedding: (None, None, 52, 30)
Layer dropout_5: (None, None, 52, 30)
Layer Convolution: (None, None, 52, 30)
Layer Maxpool: (None, None, 1, 30)
Layer words_input: (None, None)
Layer casing_input: (None, None)
Layer Flatten: (None, None, 30)
Layer lambda_3: (None, None, 1024)
Layer embedding_6: (None, None, 8)
Layer dropout_6: (None, None, 30)
Layer concatenate_3: (None, None, 1062)
Layer BLSTM: (None, None, 550)
Layer Softmax_layer: (None, None, 9)
Model built. Saved model.png

Epoch 0/80
Batch len: dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '62', '67', '78', '113'])
f1 test  0
f1 dev  0 

Epoch 1/80
Batch len: dict_

TypeError: ignored

In [0]:
sentences = [ 'he said further scientific study was required and if it was found that action was needed it should be taken by the european union .'
 ,'he said no scientific study was required and if it was found that action was needed it should be taken by the european union .']
sentences = np.asarray(sentences)
# res = tf.reshape(sentences, [-1])
print(res)

Tensor("Reshape:0", shape=(2,), dtype=string)


In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
embeddings = elmo(tf.squeeze(tf.cast(sentences, tf.string)),
signature="default",
as_dict=True)["elmo"]


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0330 19:08:53.286401 140299740088192 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
tf.squeeze(tf.cast(sentences, tf.string))

<tf.Tensor 'Squeeze_1:0' shape=(2,) dtype=string>

In [0]:
print(embeddings)

Tensor("module_4_apply_default/aggregation/mul_3:0", shape=(2, 25, 1024), dtype=float32)


In [0]:
from keras.datasets import imdb

def load_data(vocab_size, max_len):
    """
        Loads the keras imdb dataset

        Args:
            vocab_size = {int} the size of the vocabulary
            max_len = {int} the maximum length of input considered for padding

        Returns:
            X_train = tokenized train data
            X_test = tokenized test data

    """
    INDEX_FROM = 3

    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size, index_from=INDEX_FROM)

    return X_train, X_test, y_train, y_test

Using TensorFlow backend.


In [0]:
    x_train,x_test,y_train,y_test = load_data(10000, 200)


17465344/17464789 [==============================] - 0s 0us/step


In [0]:
import numpy as np
def data_prep_ELMo(train_x, train_y, test_x, test_y, max_len):
    INDEX_FROM = 3
    train_text,train_label,test_text,test_label = data_prep_ELMo(x_train,y_train,x_test,y_test,200)
    word_to_index = {k: (v + INDEX_FROM) for k, v in word_to_index.items()}

    word_to_index["<START>"] = 1
    word_to_index["<UNK>"] = 2

    index_to_word = {v: k for k, v in word_to_index.items()}

    sentences = []
    for i in range(len(train_x)):
        temp = [index_to_word[ids] for ids in train_x[i]]
        sentences.append(temp)

    test_sentences = []
    for i in range(len(test_x)):
        temp = [index_to_word[ids] for ids in test_x[i]]
        test_sentences.append(temp)

        
    print("first: {}".format(' '.join(sentences[0][:max_len])))
    train_text = [' '.join(sentences[i][:max_len]) for i in range(len(sentences))]
    train_text = np.array(train_text, dtype=object)[:, np.newaxis]
    train_label = train_y.tolist()

    test_text = [' '.join(test_sentences[i][:500]) for i in range(len(test_sentences))]
    test_text = np.array(test_text, dtype=object)[:, np.newaxis]
    test_label = test_y.tolist()

    return train_text, train_label, test_text, test_label

In [0]:
train_text,train_label,test_text,test_label = data_prep_ELMo(x_train, y_train,x_test, y_test, 200)


RecursionError: ignored

In [0]:
train_text,train_label,test_text,test_label = data_prep_ELMo(x_train,y_train,x_test,y_test,200)


In [0]:
print(train_text[0])

["<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for w